# Déployez un modèle dans le cloud

In [1]:
from pyspark import SparkContext, SparkConf
import sagemaker_pyspark

conf = (SparkConf()
        .set("spark.driver.extraClassPath", ":".join(sagemaker_pyspark.classpath_jars())))


In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession
             .builder
             .config(conf=conf)
             .appName('P8')
             .master('local')
             .config("spark.sql.parquet.writeLegacyFormat", 'true') # OC
             .config('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem') # MC
             .appName("Fruit Image Classification")
             .getOrCreate()
)

# SparkContext(conf=conf)
sc = spark.sparkContext

print("done")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/27 13:55:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
done


In [3]:
# https://sagemaker-pyspark.readthedocs.io/en/latest/
from sagemaker_pyspark import IAMRole

iam_role = "arn:aws:iam::319752300128:role/service-role/SageMaker-DS2"

region = "eu-central-1"
# training_data = spark.read.format("libsvm").option("numFeatures", "784")
  # .load("s3a://sagemaker-sample-data-{}/spark/mnist/train/".format(region))

S3_bucket='p8-images'
data_key = 'sample'
data_location = 's3a://{}/{}'.format(S3_bucket, data_key)




# Séléction des blocs à activer

In [4]:
#Testés
seq_actions = ['lect_2', 'feat_2'] # sequence pour PCA ok
# seq_actions = ['lect_1', 'feat_1'] # OOM


# Lecture sur s3

In [5]:
import numpy as np

## Méthode de lecture 1 ("alternant")
spark.read / IAM

In [6]:
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

if 'lect_1' in seq_actions:
    # lecture des ressources via spark / sagemaker_pypsark - approche "alternant"
    images = spark.read.format("binaryFile").option("pathGlobFilter", "*.jpg").option("recursiveFileLookup", "true").load(data_location)
    # https://sagemaker-pyspark.readthedocs.io/en/latest/
    # training_data = spark.read.format("libsvm").option("numFeatures", "784")
      # .load("s3a://sagemaker-sample-data-{}/spark/mnist/train/".format(region))

    print(images)
    
    images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
    print(images.printSchema())
    print(images.select('path','label').show(5,False))

## Méthode de lecture 2
boto3 ("lect_2")

In [7]:
import pyspark.sql.functions as F

import boto3 # S3 bucket

OUT_FOLDER='Results'

print(S3_bucket)
s3 = boto3.resource('s3')
bucket=s3.Bucket(S3_bucket)

from pyspark.sql import Row

if 'lect_2' in seq_actions:
    file_iterator= bucket.objects.filter(Prefix=data_key)
    for f in file_iterator.limit(10):
        categorie=f.key.split('/')[-2:-1]
        print(f'Categorie : {categorie}, {f.key}')

    file_iterator= bucket.objects.filter(Prefix=data_key)
    for f in file_iterator.limit(10):
        categorie=f.key.split('/')[-2:-1]
        print(f'Categorie : {categorie}, {f.key}')    
    filelist=[]
    for file in file_iterator:
        filelist.append(file.key)

    # sc= spark.sparkContext

    filelist_rdd = sc.parallelize(filelist, numSlices=4)
    print (type(filelist_rdd))
    print(filelist_rdd.getNumPartitions())

    filelist_df = filelist_rdd.map(lambda x:Row(x)).toDF(['path'])
    print(type(filelist_df))
    filelist_df.printSchema()
    filelist_df.limit(10).show(truncate=False)
    df_images = (filelist_df.withColumn('categorie',F.element_at(F.split(filelist_df['path'],"/"),2)))
    df_images.show(5,False)
    df_images.printSchema()

p8-images
Categorie : ['Kumquats'], sample/Kumquats/r_312_100.jpg
Categorie : ['Kumquats'], sample/Kumquats/r_313_100.jpg
Categorie : ['Kumquats'], sample/Kumquats/r_314_100.jpg
Categorie : ['Kumquats'], sample/Kumquats/r_3_100.jpg
Categorie : ['Kumquats'], sample/Kumquats/r_4_100.jpg
Categorie : ['Nectarine'], sample/Nectarine/r_92_100.jpg
Categorie : ['Nectarine'], sample/Nectarine/r_93_100.jpg
Categorie : ['Nectarine'], sample/Nectarine/r_94_100.jpg
Categorie : ['Nectarine'], sample/Nectarine/r_95_100.jpg
Categorie : ['Nectarine'], sample/Nectarine/r_96_100.jpg
Categorie : ['Kumquats'], sample/Kumquats/r_312_100.jpg
Categorie : ['Kumquats'], sample/Kumquats/r_313_100.jpg
Categorie : ['Kumquats'], sample/Kumquats/r_314_100.jpg
Categorie : ['Kumquats'], sample/Kumquats/r_3_100.jpg
Categorie : ['Kumquats'], sample/Kumquats/r_4_100.jpg
Categorie : ['Nectarine'], sample/Nectarine/r_92_100.jpg
Categorie : ['Nectarine'], sample/Nectarine/r_93_100.jpg
Categorie : ['Nectarine'], sample/Necta

<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- path: string (nullable = true)

+-----------------------------+
|path                         |
+-----------------------------+
|sample/Kumquats/r_312_100.jpg|
|sample/Kumquats/r_313_100.jpg|
|sample/Kumquats/r_314_100.jpg|
|sample/Kumquats/r_3_100.jpg  |
|sample/Kumquats/r_4_100.jpg  |
|sample/Nectarine/r_92_100.jpg|
|sample/Nectarine/r_93_100.jpg|
|sample/Nectarine/r_94_100.jpg|
|sample/Nectarine/r_95_100.jpg|
|sample/Nectarine/r_96_100.jpg|
+-----------------------------+

+-----------------------------+---------+
|path                         |categorie|
+-----------------------------+---------+
|sample/Kumquats/r_312_100.jpg|Kumquats |
|sample/Kumquats/r_313_100.jpg|Kumquats |
|sample/Kumquats/r_314_100.jpg|Kumquats |
|sample/Kumquats/r_3_100.jpg  |Kumquats |
|sample/Kumquats/r_4_100.jpg  |Kumquats |
+-----------------------------+---------+
only showing top 5 rows

root
 |-- path: string (nullable = true)
 |-- categorie: string 

# Featurisation 

## Méthode 1 ("alternant")

In [8]:
# conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

In [9]:
from pyspark.sql.functions import udf
from pyspark.ml.linalg import VectorUDT, Vectors
from pyspark.ml.feature import PCA

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
import pandas as pd
import numpy as np


# sc = spark.sparkContext

def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

# @udf(returnType=VectorUDT())
# def featurize_udf1(content_series_iter):
#     for content_series in content_series_iter:
#       content_series = pd.Series(content_series.tolist())
#       yield featurize_series(model, content_series)

from tensorflow.keras.applications import vgg16

def create_model():
    # vgg16.VGG16
    # MobileNetV2
    model = vgg16.VGG16(weights='imagenet',
                        include_top=True,
                        input_shape=(3, 224, 224) #sagemaker vgg16
                        # input_shape=(224, 224, 3) # colab
                        )

    for layer in model.layers:
        layer.trainable = False

    return model

# @udf(returnType=VectorUDT())
def featurize(content):
    # Load the model weights inside the UDF
    model = create_model()
    # Load the model weights
    # model.load_weights(model_weights.value)
    model.layers.pop()  # Remove the last layer
    model = Model(inputs=model.input, outputs=model.layers[-1].output)

    features = featurize_series(model, content)
    del model
    return pd.Series([Vectors.dense(f) for f in features])

featurize_RN_udf = udf(featurize, VectorUDT())

if 'feat_1' in seq_actions:
    model = create_model()
    model_weights = sc.broadcast(model.get_weights())
    
    # image_features = images.repartition(8000).select(col("path"), col("label"), featurize_RN_udf(col("content")).alias("features_images"))
    image_features = images.repartition(15000).select(col("path"), col("label"), featurize_RN_udf(col("content")).alias("features_images"))
    
    image_features.printSchema()   
    
    # udf_vectorized = udf(lambda r: Vectors.dense(r), VectorUDT())
    # image_features = image_features.repartition(15000).withColumn("vector_data", )
    # image_features = image_features.repartition(15000).select(col("path"), col("label"), udf_vectorized(F.col("features_images")).alias("features_images"))
    # image_features.printSchema()


In [10]:
if 'feat_1' in seq_actions:
    
    from pyspark.ml.feature import PCA
    from pyspark.ml import Pipeline
    from pyspark.ml.feature import VectorAssembler
    from pyspark.ml.feature import StandardScaler

    assembler = VectorAssembler(inputCols=['features_images'], outputCol='features_images_2')
    # assembler.transform(image_features)
 
    scaler = StandardScaler(inputCol='features_images_2', outputCol='scaled_features', withStd=True, withMean=True)
    # model = scaler.fit(image_features)
    # df_scaled = model.transform(image_features)

# Réduction de dimensions
    # pca = PCA(k=30, inputCol='scaled_features', outputCol='pca_features')
    # pca_model = pca.fit(df_scaled)
    # df_pca_features = pca_model.transform(df_scaled)
    # df_pca_features.show()

    pca = PCA(k=2, inputCol='scaled_features', outputCol='pcaFeature')
    # pipeline = Pipeline(stages=[scaler,pca])
    pipeline = Pipeline(stages=[assembler,scaler,pca])
    model = pipeline.fit(image_features)
    prediction = model.transform(image_features)


L'opération de normalisation est acceptée, mais elle s'arrête pour un problème qui s'associe à une insuffisance de mémoire :
"Answer from Java side is empty" -> 
https://stackoverflow.com/questions/37252809/apache-spark-py4jerror-answer-from-java-side-is-empty#40803994
    "Usually, you'll see this error when the Java process get silently killed by the OOM Killer."
https://stackoverflow.com/questions/21138751/spark-java-lang-outofmemoryerror-java-heap-space/22742982#22742982
    "Try using more partitions, you should have 2 - 4 per CPU. IME increasing the number of partitions is often the easiest way to make a program more stable (and often faster). For huge amounts of data you may need way more than 4 per CPU, I've had to use 8000 partitions in some cases!"

Cette approche semble donc être correcte, mais nécessiter d'une configuration de niveau avancé pour aboutir.


## Méthode 2

In [11]:
from pyspark.sql.functions import udf
from pyspark.ml.linalg import VectorUDT, Vectors
from PIL import Image

def load_binary(file_key):
    """generic method to get any object from S3"""
    s3 = boto3.resource('s3')
    bucket= s3.Bucket(S3_bucket)
    #print('file_key',file_key)
    obj = bucket.Object(key=file_key)
    response=obj.get()
    im =response['Body']
    
    img =Image.open(im).resize([16,16])
    
    return np.array(img).flatten().tolist()

if 'feat_2' in seq_actions:
    udf_binary= F.udf(load_binary)
    df_images = df_images.withColumn('binary_data', udf_binary(F.col('path')))
    udf_vectorized = udf(lambda r: Vectors.dense(r), VectorUDT())
    df_images = df_images.withColumn("vector_data", udf_vectorized(F.col("binary_data")))
    df_images.printSchema()

root
 |-- path: string (nullable = true)
 |-- categorie: string (nullable = true)
 |-- binary_data: string (nullable = true)
 |-- vector_data: vector (nullable = true)



# PCA

In [12]:
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import PCA


if 'feat_2' in seq_actions:
    df_features=df_images.select(col('path'),col('categorie'),col('vector_data').alias('features'))
    
    # # Normalisation
    scaler = StandardScaler(inputCol='features', outputCol='scaled_features', withStd=True, withMean=True)
    model = scaler.fit(df_features)
    df_scaled = model.transform(df_features)

    # # Réduction de dimensions
    pca = PCA(k=30, inputCol='scaled_features', outputCol='pca_features')
    pca_model = pca.fit(df_scaled)
    df_pca_features = pca_model.transform(df_scaled)
    df_pca_features.show()

23/06/27 13:56:44 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/06/27 13:56:44 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


23/06/27 13:56:49 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/06/27 13:56:49 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


+--------------------+-----------+--------------------+--------------------+--------------------+
|                path|  categorie|            features|     scaled_features|        pca_features|
+--------------------+-----------+--------------------+--------------------+--------------------+
|sample/Kumquats/r...|   Kumquats|[255.0,255.0,255....|[0.46349236760188...|[-30.166743432983...|
|sample/Kumquats/r...|   Kumquats|[255.0,255.0,255....|[0.46349236760188...|[-30.314354030257...|
|sample/Kumquats/r...|   Kumquats|[255.0,255.0,255....|[0.46349236760188...|[-30.675945989220...|
|sample/Kumquats/r...|   Kumquats|[255.0,255.0,255....|[0.46349236760188...|[-28.206877298210...|
|sample/Kumquats/r...|   Kumquats|[255.0,255.0,255....|[0.46349236760188...|[-28.507394301920...|
|sample/Nectarine/...|  Nectarine|[255.0,255.0,255....|[0.46349236760188...|[24.6217297604019...|
|sample/Nectarine/...|  Nectarine|[255.0,255.0,255....|[0.46349236760188...|[25.0600587856122...|
|sample/Nectarine/..

### Sauvegarde résultat su s3

In [13]:
import io

def write_dataframe_to_csv_on_s3(dataframe, bucket, filename):
    """ Write a dataframe to a CSV on S3 """
    # Create buffer
    csv_buffer = io.StringIO()
    # Write dataframe to buffer
    dataframe.to_csv(csv_buffer, sep=",", header=None,index=None)
    # Create S3 object
    s3_resource = boto3.resource("s3")
    # Write buffer to S3 object
    s3_resource.Object(bucket, filename).put(Body=csv_buffer.getvalue())
    print(f'Writing {len(dataframe)} records to {filename} on bucket s3a://{bucket}')

if 'feat_2' in seq_actions:
    df_pandas = df_pca_features.toPandas()
    df_pandas.head()
    write_dataframe_to_csv_on_s3(df_pandas, S3_bucket, f'{OUT_FOLDER}/resultats_pca.csv')

Writing 25 records to Results/resultats_pca.csv on bucket s3a://p8-images


### PCA: variance expliquée

In [14]:
import matplotlib.pyplot as plt

def display_exppvar_plot(pca_: PCA, label: str = None):
    """
    Display a plot with cumulative explained variance.
    Parameters
    ----------
    pca_    fitted PCA decomposition
    """
    ### """Display a scree plot for the pca"""
    scree_ = pca_.explainedVariance * 100
    #print(np.round(np.cumsum(scree_), 1))
    plt.bar(np.arange(len(scree_)) + 1, scree_,)
    plt.plot(np.arange(len(scree_)) + 1,
             scree_.cumsum(), color='r', marker='o', label=label)
    # plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
    plt.title('Scree Plot of PCA: Component Eigenvalues')
    plt.axhline(y=95,linewidth=0.5, color='k', ls='--', label='95%')
    plt.xlabel("Number of principal components")
    plt.ylabel("Percentage explained variance")
    plt.title("Scree plot")
    
# set feat_1
# display_exppvar_plot(pca_model)